In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from pandas import Series, DataFrame
import os
from tqdm import tnrange, tqdm_notebook
import re

In [2]:
# csv 파일 읽어서 변수로 가져오기2 : 2019-10-01
my_location_list = []
myli = []
j = 1
member = ["김효성","신희국","이승재","한성화"]

while j == 1:
    
    try:
        my_name = input("이름을 입력하세요 : ")

        if member.index(my_name)+1 > 0:
            while j == 1:
                try:
                    goal_count = int(input("게시물 목표 갯수를 숫자로 입력하세요 : "))
                    j = 0
                except ValueError:
                    print("숫자로 입력하세요")
    except ValueError:
        print("이름이 없습니다.")

df = pd.read_csv('Desktop/read/administrative_3.csv','r', encoding ='euc-kr', delimiter=',')

my = df[df['담당자'].isin([my_name])]
my_location_list = my.loc[2:,'자치구']

for s in my_location_list:
    myli.append(s)
    
pbar = myli
bar_total = tqdm_notebook(pbar)

for char in bar_total:
    time.sleep(0.1)
    bar_total.set_description('담당지역: %s' % char)
    
    if char == myli[-1]:
        bar_total.set_description('담당지역: %s' % '로딩완료')
    
for myli_check in my_location_list:
    print(myli_check)

이름을 입력하세요 : 신희국
게시물 목표 갯수를 숫자로 입력하세요 : 50000



동대문구
중랑구
성북구
강북구
노원구
은평구


In [3]:
# 목표 갯수 이상의 지역 제외시키기

success_file = []
print_cl = []
result = []
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
path = "./Desktop/save/"
file_list = os.listdir(path)

for search_word in my_location_list:
    file_list_py = [file for file in file_list if file.startswith(search_word)]
   
    # 저장한 인스타그램 url csv 파일의 결과물 갯수 확인하기
    for f in file_list_py:
        df = pd.read_csv(path + f,'r', encoding='utf-8', delimiter=',')

        # 목표 갯수 이상으로 성공한 csv
        if len(df) > goal_count:
            print_cl.append([f, "★" + str(len(df))])
            print_c = [f, "★" + str(len(df))]
            success_file.append(f)
    
            print (search_word, ": {}".format(print_c))

    for w in success_file:
        result.append(hangul.sub('', w))

    # 성공한 지역 제외 시키기
    s = set(result)
    temp = [x for x in list(my_location_list) if x not in s]

# 앞으로 크롤링을 할 지역
print("앞으로 남은 지역 :", temp)

앞으로 남은 지역 : ['동대문구', '중랑구', '성북구', '강북구', '노원구', '은평구']


In [ ]:
# 내 지역 검색하기 : 2019-10-01

# 게시물 URL 리스트
serch_url = []

chrome_driver = 'C:/chromedriver'
driver = webdriver.Chrome(chrome_driver)

# 인스타그램 지역 키워드 검색
for search_word in temp:
    url = 'https://www.instagram.com/explore/tags/' + search_word
    resp = requests.get(url)
    driver.get(url)

    time.sleep(5)
    
    src = driver.page_source
    soup = BeautifulSoup(src)

    # 키워드 검색 전체 게시물 수
    post_count = soup.find('span',class_='g47SY')
    pst_cnt = post_count.text

    # 천단위 구분 쉼표(,) 제거
    pst_cnt = pst_cnt.replace(',', '')

    # 인트형 10진수로 변환
    pst_cnt = int(pst_cnt, base=10)

    while True:

        # 스크롤의 현재 높이
        last_height = driver.execute_script("return document.body.scrollHeight")

        # 스크롤 밑으로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 밑으로 내린 스크롤의 높이
        new_height = driver.execute_script("return document.body.scrollHeight")

        time.sleep(1)

        src = driver.page_source
        soup = BeautifulSoup(src)
        a = soup.find_all('div',class_='Nnq7C weEfm')

        for j in a:
            for i in j.find_all('a'):
                if i.get('href') not in serch_url:
                    serch_url.append(i.get('href'))
                else:
                    continue

        time.sleep(0.5)
        if new_height == last_height:

            last_height2 = driver.execute_script("return document.body.scrollHeight")

            if last_height == last_height2:
                driver.execute_script("window.scrollTo(0,200)") 
                time.sleep(0.5)
                driver.execute_script("window.scrollTo(0,400)") 
                time.sleep(0.5)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            else:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                time.sleep(2)

        # 목표 갯수를 초과할 경우 종료
        elif len(serch_url) > goal_count:
            break

        # 크롤링 계속
        else:
            last_height = new_height
            continue

        if len(serch_url)%10==0:
            print(len(serch_url))        

        time.sleep(1)

    # csv 파일로 저장하기 (바탕화면/지역_날짜_시간)
    aft_now = time.localtime()
    
    fname = "%04d%02d%02d_%02d%02d%02d.csv" % (aft_now.tm_year, aft_now.tm_mon, aft_now.tm_mday, aft_now.tm_hour, aft_now.tm_min, aft_now.tm_sec)

    dataframe = pd.DataFrame(serch_url)

    dir_path = 'Desktop'
    dir_name = 'save'

    try:
        if not(os.path.isdir('Desktop/save')):
            os.makedirs(os.path.join(dir_path + '/' + dir_name + '/'))
        dataframe.to_csv("Desktop/save/" + search_word + "_" + fname, mode='w')
        
        # 리스트 초기화
        del serch_url[:]
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

120
150
900
1350
1530
1950
2010
2220
2400
2610
2670
2700
3060
3090
3270
3570
3690
3810
4650
4680
4830
5070
5190
5310
5430
5610
5730
5760
5970
6030
6090
6390
6510
6630
7350
7530
7890
8250
8310
8370
8430
8550
8610
8730
8790
8820
9030
9210
9450
9510
9570
9630
9690
9990
10260
10470
10620
10740
10860
11370
11760
11880
12000
12120
12420
13500
13620
14100
14400
15000
15120
15570
15780
16170
16350
16530
16710
16890
16890
17190
17370
17640
17700
17820
18180
18570
18750
19020
19140
19230
19410
19440
19650
19740
20100
20190
20460
20550
20730
20910
21480
21600
21720
21810
22530
23130
23220
23340
23490
23670
23970
24000
24120
24240
24360
24450
24630
24840
24960
25110
25260
25350
25410
25710
25950
26010
27750
27930
28110
28290
28470
28650
28830
29010
29190
29370
29550
29730
29910
30090
31820
31820


In [5]:
# url 갯수 확인
print(len(serch_url))

17232


In [6]:
serch_url

['/p/B28Iro2AkW6/',
 '/p/B3EVyxkndFc/',
 '/p/B3B79wfl9lg/',
 '/p/B1fsGi7HTUW/',
 '/p/B16DtKml3QW/',
 '/p/B2TxXwMDwia/',
 '/p/B1VDaLEHZnn/',
 '/p/ByFo5NIDhvO/',
 '/p/B25slUcn6wO/',
 '/p/B3FOMoSnNIN/',
 '/p/B3FNjnFnVVx/',
 '/p/B3FNFqHngck/',
 '/p/B3FM1KCHNCc/',
 '/p/B3FLUNbgdz9/',
 '/p/B3FJj1bg0jI/',
 '/p/B3FI74zhoml/',
 '/p/B3FIlLhgZJz/',
 '/p/B3FFHmCpjH4/',
 '/p/B3FDvi0jEtG/',
 '/p/B3FDgJEAvwr/',
 '/p/B3FAVg-JgFR/',
 '/p/B3E_Eb6AK71/',
 '/p/B3E-4EJg7Jz/',
 '/p/B3E-jiPnoUg/',
 '/p/B3E-jLYnH0d/',
 '/p/B3E-ixenruS/',
 '/p/B3E-ifenGnj/',
 '/p/B3E8yhQhbjl/',
 '/p/B3E8FOeHToE/',
 '/p/B3E8EdGhMz3/',
 '/p/B3E4OannTvr/',
 '/p/B3E3665gw5B/',
 '/p/B3E36oWgujK/',
 '/p/B3E3RmQnSS6/',
 '/p/B3E3ATfFTly/',
 '/p/B3ExUClp3cl/',
 '/p/B3EwwZfnwHm/',
 '/p/B3EwAAFgrtu/',
 '/p/B3Eq5lGA5tI/',
 '/p/B3EhtRjgSjA/',
 '/p/B3ATu72nc-2/',
 '/p/B3ATWvAHl0P/',
 '/p/B3AS0SUHXkF/',
 '/p/B3ASoLFHY4j/',
 '/p/B3ASgdzHFD6/',
 '/p/B3ASMmJnQV2/',
 '/p/B3ASFcxn22t/',
 '/p/B3AR_G1n57z/',
 '/p/B3AR6MxHJ-C/',
 '/p/B3AR5ouH1-F/',


In [7]:
# csv 파일로 저장하기 (바탕화면/지역_날짜_시간)
aft_now = time.localtime()
    
fname = "%04d%02d%02d_%02d%02d%02d.csv" % (aft_now.tm_year, aft_now.tm_mon, aft_now.tm_mday, aft_now.tm_hour, aft_now.tm_min, aft_now.tm_sec)

dataframe = pd.DataFrame(serch_url)

dir_path = 'Desktop'
dir_name = 'save'

try:
    if not(os.path.isdir('Desktop/save')):
        os.makedirs(os.path.join(dir_path + '/' + dir_name + '/'))
    dataframe.to_csv("Desktop/save/" + search_word + "_" + fname, mode='w')
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise